In [ ]:
import numpy as np
import os
import pandas as pd
import os
import shutil
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
#import tensorflow as tf
import torch
#import keras
#import keras.backend as K
#from keras.optimizers import SGD
import matplotlib.pyplot as plt
#from keras.models import load_model
import torch
#from torchsummary import summary
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import roc_curve, auc
from xgboost import XGBClassifier
import xgboost as xgb

In [ ]:

# Load the data

latent_space=16

x_train = np.load('/home/axh5735/projects/signal_compression/task/16/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/16'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/16'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/16'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/16'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/16'+'/yVal.npy')
# x=np.load('tmp/old/xTrain.npy')
# y=np.load('tmp/old/yTrain.npy').reshape((x.shape[0],1))



# # Assuming you have your features (X) and labels (y)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))
# Convert y_train to integers
y_train = y_train.astype(int)

# Calculate class distribution
class_counts = np.bincount(y_train.flatten())
class_labels = np.unique(y_train)
class_distribution = dict(zip(class_labels, class_counts))

# Plot class distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(class_labels)
plt.show()

# Calculate class weights manually
total_samples = len(y_train)
class_weights = {class_label: total_samples / count for class_label, count in class_distribution.items()}

print(class_weights)

# Preprocess the data (scaling) //Changes made by Neel
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))

# Create an XGBoost classifier with class weights
xgb_classifier = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=75 / 25 , # Set scale_pos_weight based on class weights
    device='gpu')

# Train the classifier
xgb_classifier.fit(x_train_scaled, y_train.ravel())

# # You can continue with validation and testing as before

# Evaluate on the validation set

accuracy_valid = xgb_classifier.score(x_valid_scaled, y_valid)
print("Validation Accuracy:", accuracy_valid)

# Evaluate on the test set
accuracy_test = xgb_classifier.score(x_test_scaled, y_test)
print("Test Accuracy:", accuracy_test)

# Predict labels for the test set
y_pred = xgb_classifier.predict(x_test_scaled)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Define class labels (modify as needed)
class_labels = ["No Seizure", "Seizure"]

annot_kws = {"fontsize": 22, "fontweight": "bold"}
# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels,  annot_kws=annot_kws)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix 16D Latent Space')
plt.savefig('CM_16.jpeg',dpi=1200)
plt.show()

In [ ]:
############onchip

# Load the data

latent_space=16

x_train = np.load('/home/axh5735/projects/signal_compression/task/16/onchip/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/16/onchip'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/16/onchip'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/16/onchip'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/16/onchip'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/32/onchip'+'/yVal.npy')
# x=np.load('tmp/old/xTrain.npy')
# y=np.load('tmp/old/yTrain.npy').reshape((x.shape[0],1))



# # Assuming you have your features (X) and labels (y)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))
# Convert y_train to integers
y_train = y_train.astype(int)

# Calculate class distribution
class_counts = np.bincount(y_train.flatten())
class_labels = np.unique(y_train)
class_distribution = dict(zip(class_labels, class_counts))

# Plot class distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(class_labels)
plt.show()

# Calculate class weights manually
total_samples = len(y_train)
class_weights = {class_label: total_samples / count for class_label, count in class_distribution.items()}

print(class_weights)

# Preprocess the data (scaling) //Changes made by Neel
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))

# Create an XGBoost classifier with class weights
xgb_classifier = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=75 / 25 , # Set scale_pos_weight based on class weights
    device='gpu')

# Train the classifier
xgb_classifier.fit(x_train_scaled, y_train.ravel())

# # You can continue with validation and testing as before

# Evaluate on the validation set

accuracy_valid = xgb_classifier.score(x_valid_scaled, y_valid)
print("Validation Accuracy:", accuracy_valid)

# Evaluate on the test set
accuracy_test = xgb_classifier.score(x_test_scaled, y_test)
print("Test Accuracy:", accuracy_test)

# Predict labels for the test set
y_pred = xgb_classifier.predict(x_test_scaled)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Define class labels (modify as needed)
class_labels = ["No Seizure", "Seizure"]

annot_kws = {"fontsize": 22, "fontweight": "bold"}
# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels,  annot_kws=annot_kws)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix 16D Latent Space')
plt.savefig('CM_16.jpeg',dpi=1200)
plt.show()

In [ ]:

# Load the data

latent_space=32

x_train = np.load('/home/axh5735/projects/signal_compression/task/32/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/32'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/32'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/32'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/32'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/32'+'/yVal.npy')


# x=np.load('tmp/old/xTrain.npy')
# y=np.load('tmp/old/yTrain.npy').reshape((x.shape[0],1))



# # Assuming you have your features (X) and labels (y)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled_32 = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))
# Convert y_train to integers
y_train = y_train.astype(int)

# Calculate class distribution
class_counts = np.bincount(y_train.flatten())
class_labels = np.unique(y_train)
class_distribution = dict(zip(class_labels, class_counts))

# Plot class distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(class_labels)
plt.show()

# Calculate class weights manually
total_samples = len(y_train)
class_weights = {class_label: total_samples / count for class_label, count in class_distribution.items()}

print(class_weights)

# Preprocess the data (scaling) //Changes made by Neel
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))

# Create an XGBoost classifier with class weights
xgb_classifier_32 = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=75 / 25 , # Set scale_pos_weight based on class weights
    device='gpu'
)

# Train the classifier
xgb_classifier_32.fit(x_train_scaled, y_train.ravel())

# # You can continue with validation and testing as before

# Evaluate on the validation set

accuracy_valid = xgb_classifier_32.score(x_valid_scaled, y_valid)

print("Validation Accuracy:", accuracy_valid)

# Evaluate on the test set
accuracy_test = xgb_classifier_32.score(x_test_scaled_32, y_test)
print("Test Accuracy:", accuracy_test)


# Predict labels for the test set
y_pred = xgb_classifier_32.predict(x_test_scaled_32)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Define class labels (modify as needed)
class_labels = ["No Seizure", "Seizure"]

annot_kws = {"fontsize": 22, "fontweight": "bold"}
# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels,  annot_kws=annot_kws)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix 32D Latent Space')
plt.savefig('CM_32.jpeg',dpi=1200)
plt.show()

In [ ]:
import pickle
# Load the data
###on the chip
import joblib
latent_space=32

x_train = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32'+'/yVal.npy')


# x=np.load('tmp/old/xTrain.npy')
# y=np.load('tmp/old/yTrain.npy').reshape((x.shape[0],1))



# # Assuming you have your features (X) and labels (y)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled_32 = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))
# Convert y_train to integers
y_train = y_train.astype(int)

# Calculate class distribution
class_counts = np.bincount(y_train.flatten())
class_labels = np.unique(y_train)
class_distribution = dict(zip(class_labels, class_counts))

# Plot class distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(class_labels)
plt.show()

# Calculate class weights manually
total_samples = len(y_train)
class_weights = {class_label: total_samples / count for class_label, count in class_distribution.items()}

print(class_weights)

# Preprocess the data (scaling) //Changes made by Neel
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))

# Create an XGBoost classifier with class weights
xgb_classifier_32 = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=75 / 25 , # Set scale_pos_weight based on class weights
    device='gpu'
)

# Train the classifier
xgb_classifier_32.fit(x_train_scaled, y_train.ravel())

# # You can continue with validation and testing as before


# Define the file path where you want to save the model
model_filename = "/home/axh5735/projects/signal_compression/xgb_classifier_32_model5.pkl"

# Save the model to disk
#joblib.dump(xgb_classifier_32, model_filename)
with open(model_filename, 'wb') as f:
        pickle.dump(xgb_classifier_32, f)


# Evaluate on the validation set

accuracy_valid = xgb_classifier_32.score(x_valid_scaled, y_valid)

print("Validation Accuracy:", accuracy_valid)

# Evaluate on the test set
accuracy_test = xgb_classifier_32.score(x_test_scaled_32, y_test)
print("Test Accuracy:", accuracy_test)


# Predict labels for the test set
y_pred = xgb_classifier_32.predict(x_test_scaled_32)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Define class labels (modify as needed)
class_labels = ["No Seizure", "Seizure"]

annot_kws = {"fontsize": 22, "fontweight": "bold"}
# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels,  annot_kws=annot_kws)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix 32D Latent Space')
plt.savefig('CM_32.jpeg',dpi=1200)
plt.show()

## Inference and load

In [ ]:
import joblib

# Define the file path where the model is saved
model_filename = "xgb_classifier_32_model.pkl"

# Load the model from disk
loaded_model = joblib.load(model_filename)

accuracy_test = loaded_model.score(x_test_scaled_32, y_test)
print("Test Accuracy:", accuracy_test)


In [ ]:

# Load the data

latent_space=64

x_train = np.load('/home/axh5735/projects/signal_compression/task/64/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/64'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/64'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/64'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/64'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/64'+'/yVal.npy')

# x=np.load('tmp/old/xTrain.npy')
# y=np.load('tmp/old/yTrain.npy').reshape((x.shape[0],1))



# # Assuming you have your features (X) and labels (y)
# x_train, x_t, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled_64 = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))
# Convert y_train to integers
y_train = y_train.astype(int)

# Calculate class distribution
class_counts = np.bincount(y_train.flatten())
class_labels = np.unique(y_train)
class_distribution = dict(zip(class_labels, class_counts))

# Plot class distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(class_labels)
plt.show()

# Calculate class weights manually
total_samples = len(y_train)
class_weights = {class_label: total_samples / count for class_label, count in class_distribution.items()}

print(class_weights)

# Preprocess the data (scaling) //Changes made by Neel
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))

# Create an XGBoost classifier with class weights
xgb_classifier_64 = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=75 / 25 , # Set scale_pos_weight based on class weights
    device='gpu'
    
)

# Train the classifier
xgb_classifier_64.fit(x_train_scaled, y_train.ravel())

# # You can continue with validation and testing as before

# Evaluate on the validation set

accuracy_valid = xgb_classifier_64.score(x_valid_scaled, y_valid)

print("Validation Accuracy:", accuracy_valid)

# Evaluate on the test set
accuracy_test = xgb_classifier_64.score(x_test_scaled_64, y_test)
print("Test Accuracy:", accuracy_test)


# Predict labels for the test set
y_pred = xgb_classifier_64.predict(x_test_scaled_64)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Define class labels (modify as needed)
class_labels = ["No Seizure", "Seizure"]

annot_kws = {"fontsize": 22, "fontweight": "bold"}
# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels,  annot_kws=annot_kws)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix 64D Latent Space')
plt.savefig('CM_64.jpeg',dpi=1200)
plt.show()

In [ ]:
# Load the data

#####on the chip

latent_space=64

x_train = np.load('/home/axh5735/projects/signal_compression/task/64/onchip/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/64/onchip'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/64/onchip'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/64/onchip'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/64/onchip'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/64/onchip'+'/yVal.npy')

# x=np.load('tmp/old/xTrain.npy')
# y=np.load('tmp/old/yTrain.npy').reshape((x.shape[0],1))



# # Assuming you have your features (X) and labels (y)
# x_train, x_t, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled_64 = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))
# Convert y_train to integers
y_train = y_train.astype(int)

# Calculate class distribution
class_counts = np.bincount(y_train.flatten())
class_labels = np.unique(y_train)
class_distribution = dict(zip(class_labels, class_counts))

# Plot class distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(class_labels)
plt.show()

# Calculate class weights manually
total_samples = len(y_train)
class_weights = {class_label: total_samples / count for class_label, count in class_distribution.items()}

print(class_weights)

# Preprocess the data (scaling) //Changes made by Neel
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))

# Create an XGBoost classifier with class weights
xgb_classifier_64 = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=75 / 25 , # Set scale_pos_weight based on class weights
    device='gpu'
    
)

# Train the classifier
xgb_classifier_64.fit(x_train_scaled, y_train.ravel())

# # You can continue with validation and testing as before

# Evaluate on the validation set

accuracy_valid = xgb_classifier_64.score(x_valid_scaled, y_valid)

print("Validation Accuracy:", accuracy_valid)

# Evaluate on the test set
accuracy_test = xgb_classifier_64.score(x_test_scaled_64, y_test)
print("Test Accuracy:", accuracy_test)


# Predict labels for the test set
y_pred = xgb_classifier_64.predict(x_test_scaled_64)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Define class labels (modify as needed)
class_labels = ["No Seizure", "Seizure"]

annot_kws = {"fontsize": 22, "fontweight": "bold"}
# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels,  annot_kws=annot_kws)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix 64D Latent Space')
plt.savefig('CM_64.jpeg',dpi=1200)
plt.show()

In [ ]:

# Load the data

latent_space=128

x_train = np.load('/home/axh5735/projects/signal_compression/task/128/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/128'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/128'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/128'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/128'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/128'+'/yVal.npy')



# x=np.load('tmp/old/xTrain.npy')
# y=np.load('tmp/old/yTrain.npy').reshape((x.shape[0],1))



# # Assuming you have your features (X) and labels (y)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled_128 = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))
# Convert y_train to integers
y_train = y_train.astype(int)

# Calculate class distribution
class_counts = np.bincount(y_train.flatten())
class_labels = np.unique(y_train)
class_distribution = dict(zip(class_labels, class_counts))

# Plot class distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(class_labels)
plt.show()

# Calculate class weights manually
total_samples = len(y_train)
class_weights = {class_label: total_samples / count for class_label, count in class_distribution.items()}

print(class_weights)

# Preprocess the data (scaling) //Changes made by Neel
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))

# Create an XGBoost classifier with class weights
xgb_classifier_128 = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=75 / 25 , # Set scale_pos_weight based on class weights
    device='gpu'
    #tree_method='gpu_hist'
)

# Train the classifier
xgb_classifier_128.fit(x_train_scaled, y_train.ravel())

# # You can continue with validation and testing as before

# Evaluate on the validation set

accuracy_valid = xgb_classifier_128.score(x_valid_scaled, y_valid)

print("Validation Accuracy:", accuracy_valid)

# Evaluate on the test set
accuracy_test = xgb_classifier_128.score(x_test_scaled_128, y_test)
print("Test Accuracy:", accuracy_test)


# Predict labels for the test set
y_pred = xgb_classifier_128.predict(x_test_scaled_128)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Define class labels (modify as needed)
class_labels = ["No Seizure", "Seizure"]

annot_kws = {"fontsize": 22, "fontweight": "bold"}
# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels,  annot_kws=annot_kws)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix 128D Latent Space')
plt.savefig('CM_128.jpeg',dpi=1200)
plt.show()





In [ ]:
# Load the data
###on the chip

latent_space=128

x_train = np.load('/home/axh5735/projects/signal_compression/task/128/onchip/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/128/onchip'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/128/onchip'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/128/onchip'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/128/onchip'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/128/onchip'+'/yVal.npy')



# x=np.load('tmp/old/xTrain.npy')
# y=np.load('tmp/old/yTrain.npy').reshape((x.shape[0],1))



# # Assuming you have your features (X) and labels (y)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled_128 = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))
# Convert y_train to integers
y_train = y_train.astype(int)

# Calculate class distribution
class_counts = np.bincount(y_train.flatten())
class_labels = np.unique(y_train)
class_distribution = dict(zip(class_labels, class_counts))

# Plot class distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(class_labels)
plt.show()

# Calculate class weights manually
total_samples = len(y_train)
class_weights = {class_label: total_samples / count for class_label, count in class_distribution.items()}

print(class_weights)

# Preprocess the data (scaling) //Changes made by Neel
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))

# Create an XGBoost classifier with class weights
xgb_classifier_128 = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=75 / 25 , # Set scale_pos_weight based on class weights
    device='gpu'
    #tree_method='gpu_hist'
)

# Train the classifier
xgb_classifier_128.fit(x_train_scaled, y_train.ravel())

# # You can continue with validation and testing as before

# Evaluate on the validation set

accuracy_valid = xgb_classifier_128.score(x_valid_scaled, y_valid)

print("Validation Accuracy:", accuracy_valid)

# Evaluate on the test set
accuracy_test = xgb_classifier_128.score(x_test_scaled_128, y_test)
print("Test Accuracy:", accuracy_test)


# Predict labels for the test set
y_pred = xgb_classifier_128.predict(x_test_scaled_128)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Define class labels (modify as needed)
class_labels = ["No Seizure", "Seizure"]

annot_kws = {"fontsize": 22, "fontweight": "bold"}
# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels,  annot_kws=annot_kws)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix 128D Latent Space')
plt.savefig('CM_128.jpeg',dpi=1200)
plt.show()




In [ ]:

# Load the data

latent_space=256

x_train = np.load('/home/axh5735/projects/signal_compression/task/256/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/256'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/256'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/256'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/256'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/256'+'/yVal.npy')


# x=np.load('tmp/old/xTrain.npy')
# y=np.load('tmp/old/yTrain.npy').reshape((x.shape[0],1))



# # Assuming you have your features (X) and labels (y)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled_256 = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))
# Convert y_train to integers
y_train = y_train.astype(int)

# Calculate class distribution
class_counts = np.bincount(y_train.flatten())
class_labels = np.unique(y_train)
class_distribution = dict(zip(class_labels, class_counts))

# Plot class distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(class_labels)
plt.show()

# Calculate class weights manually
total_samples = len(y_train)
class_weights = {class_label: total_samples / count for class_label, count in class_distribution.items()}

print(class_weights)

# Preprocess the data (scaling) //Changes made by Neel
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))

# Create an XGBoost classifier with class weights
xgb_classifier_256 = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=75 / 25 , # Set scale_pos_weight based on class weights
    device='gpu'
)

# Train the classifier
xgb_classifier_256.fit(x_train_scaled, y_train.ravel())

# # You can continue with validation and testing as before

# Evaluate on the validation set

accuracy_valid = xgb_classifier_256.score(x_valid_scaled, y_valid)

print("Validation Accuracy:", accuracy_valid)

# Evaluate on the test set
accuracy_test = xgb_classifier_256.score(x_test_scaled_256, y_test)
print("Test Accuracy:", accuracy_test)


# Predict labels for the test set
y_pred = xgb_classifier_256.predict(x_test_scaled_256)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Define class labels (modify as needed)
class_labels = ["No Seizure", "Seizure"]

sns.set(font_scale=1.2)  # Set font scale for all text in the plot

# Increase font size and make it bold for annotations (numbers in cells)
annot_kws = {"fontsize": 22, "fontweight": "bold"}
# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels,  annot_kws=annot_kws)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix 256D Latent Space')
plt.savefig('CM_256.jpeg',dpi=1200)
plt.show()

In [ ]:
# Load the data
###on the chip

latent_space=256

x_train = np.load('/home/axh5735/projects/signal_compression/task/256/onchip/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/256/onchip'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/256/onchip'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/256/onchip'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/256/onchip'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/256/onchip'+'/yVal.npy')


# x=np.load('tmp/old/xTrain.npy')
# y=np.load('tmp/old/yTrain.npy').reshape((x.shape[0],1))



# # Assuming you have your features (X) and labels (y)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled_256 = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))
# Convert y_train to integers
y_train = y_train.astype(int)

# Calculate class distribution
class_counts = np.bincount(y_train.flatten())
class_labels = np.unique(y_train)
class_distribution = dict(zip(class_labels, class_counts))

# Plot class distribution
plt.bar(class_labels, class_counts)
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(class_labels)
plt.show()

# Calculate class weights manually
total_samples = len(y_train)
class_weights = {class_label: total_samples / count for class_label, count in class_distribution.items()}

print(class_weights)

# Preprocess the data (scaling) //Changes made by Neel
# scaler = StandardScaler()
# x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))

# Create an XGBoost classifier with class weights
xgb_classifier_256 = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=75 / 25 , # Set scale_pos_weight based on class weights
    device='gpu'
)

# Train the classifier
xgb_classifier_256.fit(x_train_scaled, y_train.ravel())

# # You can continue with validation and testing as before

# Evaluate on the validation set

accuracy_valid = xgb_classifier_256.score(x_valid_scaled, y_valid)

print("Validation Accuracy:", accuracy_valid)

# Evaluate on the test set
accuracy_test = xgb_classifier_256.score(x_test_scaled_256, y_test)
print("Test Accuracy:", accuracy_test)


# Predict labels for the test set
y_pred = xgb_classifier_256.predict(x_test_scaled_256)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Define class labels (modify as needed)
class_labels = ["No Seizure", "Seizure"]

sns.set(font_scale=1.2)  # Set font scale for all text in the plot

# Increase font size and make it bold for annotations (numbers in cells)
annot_kws = {"fontsize": 22, "fontweight": "bold"}
# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels,  annot_kws=annot_kws)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix 256D Latent Space')
plt.savefig('CM_256.jpeg',dpi=1200)
plt.show()

In [ ]:
# Load the data
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import catplot
from sklearn.metrics import roc_curve, auc

latent_space=32

x_train = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/32/onchip_32'+'/yVal.npy')
# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))

# Convert y_train and y_valid to integers
y_train_int = y_train.astype(np.int64)
y_valid_int = y_valid.astype(np.int64)

# Calculate class distribution
class_counts = np.bincount(y_train_int.flatten())
class_labels = np.unique(y_train_int)
class_distribution = dict(zip(class_labels, class_counts))

# Calculate class weights manually
total_samples = len(y_train_int)
class_weights = {class_label: total_samples / (2 * count) for class_label, count in class_distribution.items()}

# Initialize an XGBoost classifier
xgb_classifier = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=class_weights[1] / class_weights[0],  # Set scale_pos_weight based on class weights
    device='gpu',
    early_stopping_rounds=50
)

# Initialize k-fold cross-validation
k_folds = 10
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Train and validate using k-fold cross-validation
avg_valid_accuracy = 0

precision, recall,auc_score = [],[],[]
for fold, (train_idx, valid_idx) in enumerate(skf.split(x_train_scaled, y_train_int)):
    x_train_fold, x_valid_fold = x_train_scaled[train_idx], x_train_scaled[valid_idx]
    y_train_fold, y_valid_fold = y_train_int[train_idx], y_train_int[valid_idx]

    # Train the XGBoost classifier
    xgb_classifier.fit(x_train_fold, y_train_fold, eval_set=[(x_valid_fold, y_valid_fold)],
                       verbose=False)

    valid_predictions = xgb_classifier.predict(x_valid_fold)
    valid_accuracy = accuracy_score(y_valid_fold, valid_predictions)
    avg_valid_accuracy += valid_accuracy
    print(f"Fold {fold + 1} - Validation Accuracy: {valid_accuracy:.4f}")
    y_pred_prob=xgb_classifier.predict_proba(x_valid_fold)[:, 1]
    p,r,_=precision_recall_curve(y_valid_fold, y_pred_prob)
    precision.append(p)
    recall.append(r)
# Calculate area under the precision-recall curve
    auc_score.append(auc(r, p))

    



In [ ]:
import numpy as np
from scipy.interpolate import make_interp_spline

font = {'family': 'serif',
        'color':  'black',
        'weight': 'bold',
        'size': 14,
        }

# Define a list of pastel colors
pastel_colors = ['#FFB6C1', '#FFD700', '#87CEEB', '#98FB98', '#FFA07A', '#9370DB', '#00CED1', '#FF6347', '#FF69B4', '#00FA9A']

# Plot the precision-recall curve
plt.figure(figsize=(9, 6))
for i in range(10):
    # Make sure x-values are unique
    unique_recall, unique_indices = np.unique(recall[i], return_index=True)
    unique_precision = [precision[i][index] for index in unique_indices]
    
    # Smooth the recall and precision values using spline interpolation
    recall_smooth = np.linspace(unique_recall.min(), unique_recall.max(), 300)
    precision_smooth = make_interp_spline(unique_recall, unique_precision)(recall_smooth)
    if i==0:
        plt.plot(recall_smooth, precision_smooth, color=pastel_colors[i], linewidth=3, label=f'auc_score for {i+1}st fold = {auc_score[i]:.2f}')
    elif i==1:
        plt.plot(recall_smooth, precision_smooth, color=pastel_colors[i], linewidth=3, label=f'auc_score for {i+1}nd fold = {auc_score[i]:.2f}')
    elif i==2:
        plt.plot(recall_smooth, precision_smooth, color=pastel_colors[i], linewidth=3, label=f'auc_score for {i+1}rd fold = {auc_score[i]:.2f}')
    else:
        plt.plot(recall_smooth, precision_smooth, color=pastel_colors[i], linewidth=3, label=f'auc_score for {i+1}s fold = {auc_score[i]:.2f}')
plt.plot([0, 1], [0.3, 0.3], color='#CCCCCC', linestyle='--',linewidth=3 )
plt.xlabel('Recall', fontdict=font)
plt.ylabel('Precision', fontdict=font)
plt.title('Precision-Recall Curve for 32D Latent Space', fontdict=font)
plt.legend(loc='center left', fontsize=14, bbox_to_anchor=(0.03, 0.55))
plt.grid(True, alpha=0.3, linestyle='-')  # Adjust the alpha parameter for transparency
plt.ylim(0.2, 1.05)
plt.yticks(fontsize=14) 
plt.xticks(fontsize=14) 
plt.tight_layout()
plt.savefig('k-fold_PR.jpeg', dpi=1200)
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score
import catplot
from sklearn.metrics import roc_curve, auc

latent_space = 64
x_train = np.load('/home/axh5735/projects/signal_compression/task/64/onchip/xTrain.npy')
y_train = np.load('/home/axh5735/projects/signal_compression/task/64/onchip'+'/yTrain.npy')
x_test = np.load('/home/axh5735/projects/signal_compression/task/64/onchip'+'/xTest.npy')
y_test = np.load('/home/axh5735/projects/signal_compression/task/64/onchip'+'/yTest.npy')
x_valid = np.load('/home/axh5735/projects/signal_compression/task/64/onchip'+'/xVal.npy')
y_valid = np.load('/home/axh5735/projects/signal_compression/task/64/onchip'+'/yVal.npy')

# Preprocess the data (scaling)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_scaled = scaler.transform(x_test.reshape(x_test.shape[0], -1))
x_valid_scaled = scaler.transform(x_valid.reshape(x_valid.shape[0], -1))

# Convert y_train and y_valid to integers
y_train_int = y_train.astype(np.int64)
y_valid_int = y_valid.astype(np.int64)

# Calculate class distribution
class_counts = np.bincount(y_train_int.flatten())
class_labels = np.unique(y_train_int)
class_distribution = dict(zip(class_labels, class_counts))

# Calculate class weights manually
total_samples = len(y_train_int)
class_weights = {class_label: total_samples / (2 * count) for class_label, count in class_distribution.items()}

# Initialize an XGBoost classifier
xgb_classifier = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=class_weights[1] / class_weights[0],  # Set scale_pos_weight based on class weights
    tree_method='gpu_hist',
    early_stopping_rounds=50
)

# Initialize k-fold cross-validation
k_folds = 10
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Train and validate using k-fold cross-validation
avg_valid_accuracy = 0

precision, recall,auc_score = [],[],[]
for fold, (train_idx, valid_idx) in enumerate(skf.split(x_train_scaled, y_train_int)):
    x_train_fold, x_valid_fold = x_train_scaled[train_idx], x_train_scaled[valid_idx]
    y_train_fold, y_valid_fold = y_train_int[train_idx], y_train_int[valid_idx]

    # Train the XGBoost classifier
    xgb_classifier.fit(x_train_fold, y_train_fold, eval_set=[(x_valid_fold, y_valid_fold)],
                       verbose=False)

    valid_predictions = xgb_classifier.predict(x_valid_fold)
    valid_accuracy = accuracy_score(y_valid_fold, valid_predictions)
    avg_valid_accuracy += valid_accuracy
    print(f"Fold {fold + 1} - Validation Accuracy: {valid_accuracy:.4f}")
    y_pred_prob=xgb_classifier.predict_proba(x_valid_fold)[:, 1]
    #p,r,_=precision_recall_curve(y_valid_fold, y_pred_prob)
    p,r,_=roc_curve(y_valid_fold, y_pred_prob)
    precision.append(p)
    recall.append(r)
# Calculate area under the precision-recall curve
    auc_score.append(auc(p, r))


In [ ]:
import numpy as np
from scipy.interpolate import make_interp_spline

font = {'family': 'serif',
        'color':  'black',
        'weight': 'bold',
        'size': 14,
        }

# Define a list of pastel colors
pastel_colors = ['#FFB6C1', '#FFD700', '#87CEEB', '#98FB98', '#FFA07A', '#9370DB', '#00CED1', '#FF6347', '#FF69B4', '#00FA9A']

# Plot the precision-recall curve
plt.figure(figsize=(9, 6))
for i in range(10):
    # Make sure x-values are unique
    unique_recall, unique_indices = np.unique(recall[i], return_index=True)
    unique_precision = [precision[i][index] for index in unique_indices]
    
    # Smooth the recall and precision values using spline interpolation
    recall_smooth = np.linspace(unique_recall.min(), unique_recall.max(), 300)
    precision_smooth = make_interp_spline(unique_recall, unique_precision)(recall_smooth)
    if i==0:
        plt.plot(precision_smooth,recall_smooth, color=pastel_colors[i], linewidth=3, label=f'auc_score for {i+1}st fold = {auc_score[i]:.2f}')
    elif i==1:
        plt.plot(precision_smooth,recall_smooth, color=pastel_colors[i], linewidth=3, label=f'auc_score for {i+1}nd fold = {auc_score[i]:.2f}')
    elif i==2:
        plt.plot(precision_smooth,recall_smooth, color=pastel_colors[i], linewidth=3, label=f'auc_score for {i+1}rd fold = {auc_score[i]:.2f}')
    else:
        plt.plot(precision_smooth,recall_smooth, color=pastel_colors[i], linewidth=3, label=f'auc_score for {i+1}th fold = {auc_score[i]:.2f}')
plt.plot([0, 1], [0, 1], color='#CCCCCC', linestyle='--',linewidth=3 )
plt.xlabel('False Positive rate', fontdict=font)
plt.ylabel('True Positive rate', fontdict=font)
plt.title('ROC Curve for 64D Latent Space', fontdict=font)
plt.legend(loc='lower right', fontsize=14, bbox_to_anchor=(1, 0))
plt.grid(True, alpha=0.3, linestyle='-')  # Adjust the alpha parameter for transparency
plt.yticks(fontsize=14) 
plt.xticks(fontsize=14) 
plt.tight_layout()
plt.savefig('k-fold_roc.jpeg', dpi=1200)
plt.show()


In [ ]:
y_pred_prob=[]
yValPred=np.load('oldpred.npy')
yValTrue=np.load('oldtrue.npy')
y_pred_prob.append(xgb_classifier.predict_proba(x_test_scaled)[:, 1])
y_pred_prob.append(xgb_classifier_32.predict_proba(x_test_scaled_32)[:, 1])
y_pred_prob.append(xgb_classifier_64.predict_proba(x_test_scaled_64)[:, 1])
y_pred_prob.append(xgb_classifier_128.predict_proba(x_test_scaled_128)[:, 1])
y_pred_prob.append(xgb_classifier_256.predict_proba(x_test_scaled_256)[:, 1])
# Calculate precision and recall values
precision, recall,auc_score = [],[],[]
for i in range(5):
    p,r,_=precision_recall_curve(y_test, y_pred_prob[i])
    unique_recall, unique_indices = np.unique(r, return_index=True)
    unique_precision = [p[index] for index in unique_indices]
    
    # Smooth the recall and precision values using spline interpolation
    recall_smooth = np.linspace(unique_recall.min(), unique_recall.max(), 300)
    precision_smooth = make_interp_spline(unique_recall, unique_precision)(recall_smooth)
    precision.append(precision_smooth)
    recall.append(recall_smooth)
# Calculate area under the precision-recall curve
    auc_score.append( auc(r, p)) 
precision1, recall1, thresholds1 = precision_recall_curve(yValTrue, yValPred)
auc_score1 = auc(recall1, precision1)
font = {'family': 'serif',
        'color':  'black',
        'weight': 'bold',
        'size': 14,
        }
pastel_colors = ['#FFB6C1', '#FFD700', '#87CEEB', '#98FB98', '#FFA07A', '#9370DB']
# Plot the precision-recall curve
plt.figure(figsize=(9, 6))
plt.plot(recall[0], precision[0], color=pastel_colors[0],linewidth=4, label=f'AUC of 16D = {auc_score[0]:.2f}')
plt.plot(recall[1], precision[1], color=pastel_colors[1],linewidth=4, label=f'AUC of 32D = {auc_score[1]:.2f}')
plt.plot(recall[2], precision[2], color=pastel_colors[2],linewidth=4, label=f'AUC of 64D = {auc_score[2]:.2f}')
plt.plot(recall[3], precision[3], color=pastel_colors[3], linewidth=4,label=f'AUC of 128D = {auc_score[3]:.2f}')
plt.plot(recall[4], precision[4], color=pastel_colors[4], linewidth=4,label=f'AUC of 256D = {auc_score[4]:.2f}')
plt.plot(recall1, precision1, color=pastel_colors[5],linewidth=4, label=f'AUC Original = {auc_score1:.2f}')
plt.plot([0, 1], [0.3, 0.3], color='#CCCCCC', linestyle='--',linewidth=3 )
plt.xlabel('Recall',fontdict=font)
plt.ylabel('Precision',fontdict=font)
plt.title('Precision-Recall Curve for XGBoost on Seizure Detection',fontdict=font)
plt.legend(loc='center left', fontsize=14, bbox_to_anchor=(0.03, 0.60))
plt.grid(True, alpha=0.3, linestyle='-')  # Adjust the alpha parameter for transparency
plt.ylim(0.2, 1.05)
plt.yticks(fontsize=14) 
plt.xticks(fontsize=14) 
plt.tight_layout()
plt.savefig('roc-curve_cmp.jpeg',dpi=1200)
plt.show()

In [ ]:

def custom_loss(y_true, y_pred):
    epsilon = 1e-7
    
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
    
    bce_loss = (0.7 * -y_true * torch.log(y_pred)) + (0.3 * -(1 - y_true) * torch.log(1 - y_pred))
    mean_bce_loss = torch.mean(bce_loss)
    return mean_bce_loss

a=[1,1,0,0]
b=[1,1,0,0]
for i in range(5):
    threshold = 0.5  # Adjust this threshold as needed
    yValPred_binary = (y_pred_prob[i] > threshold).astype(int)
    print(custom_loss(torch.tensor(yValPred_binary),torch.tensor(y_test)))

In [ ]:
categories=['16D (1:1170 CR)', '32D (1:585 CR)', '64D (1:293 CR)', '128D (1:146 CR)', '256D (1:73 CR)']
values=[84, 86, 91, 87, 83]
colors = [(0.39, 0.58, 0.93, 0.8), (0.39, 0.58, 0.93, 0.8), (0.39, 0.58, 0.93, 0.8), (0.39, 0.58, 0.93, 0.8), (0.39, 0.58, 0.93, 0.8)]
font = {'family': 'serif',
        'color':  'black',
        'weight': 'bold',
        'size': 14,
        }

plt.figure(figsize=(10, 6))
bars = plt.bar(categories,values,color=colors)
plt.xlabel('Latent Space Dimensions (Compression ratio)',fontdict=font)
plt.ylabel('Detection Accuracy',fontdict=font)
plt.title('Compression vs Accuracy',fontdict=font)
# plt.xticks(rotation=45, ha='right')
plt.ylim(80, 93)  # Set y-axis limits
plt.yticks(range(80, 93, 1),fontsize=14) 

plt.grid(True, alpha=0.3, linestyle='-')  # Adjust the alpha parameter for transparency

plt.tight_layout()
text_gap = 0.4
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval +  text_gap, f'{round(yval)}%', ha='center', fontsize=14)
    plt.xticks(fontsize=14) 

plt.savefig('accuracy.jpeg',dpi=1200)
plt.show()